In [1]:
import tweepy
from textblob import TextBlob
import nltk
from nltk.stem import WordNetLemmatizer
import webbrowser
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
log = pd.read_csv("login.csv")
apiKey = log.value[3]
apiKeySecret = log.value[4]
accessToken = log.value[0]
accessTokenSecret = log.value[2]
callback_uri= 'oob'
auth = tweepy.OAuthHandler(apiKey,apiKeySecret,callback=callback_uri)
redirect_url = auth.get_authorization_url()
webbrowser.open(redirect_url)
user_input_input = input('pin value is -')
auth.get_access_token(user_input_input)
api = tweepy.API(auth)

In [3]:
topL = pd.read_csv("topL.csv")
topList=[]
for k in topL['Company']:
    if(len(k)==1):
        topList.append(k)
    else:
        topList.append(k.split()[0])

In [4]:
def cleanTxt(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u'\u0119'
                               u'\u011f-\u999f'
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               u'\u200b-\u900b'
                               u'\u0105'
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'',text)
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink

    return text
en_stop = set(nltk.corpus.stopwords.words('english'))
stemmer = WordNetLemmatizer() 
def pre_processing_text(doc):
    doc = re.sub(r'\W', ' ', str(doc))
    doc = re.sub(r'\s+[a-zA-Z]\s+', ' ', str(doc))
    doc = re.sub(r'\^[a-zA-Z]\s+', ' ', str(doc))
    doc = re.sub(r'\s+', ' ', str(doc), flags=re.I)
    doc = re.sub(r'^b\s+', '', str(doc))
    doc = doc.lower()
    tokens = doc.split()
    token = [stemmer.lemmatize(word=word) for word in tokens]
    token = [word for word in token if word not in en_stop]
    token = [word for word in token if len(word)>3]

    preprocess_text = ' '.join(tokens)
    return preprocess_text

In [5]:
df = pd.DataFrame(columns=['Tweets','Company'])

In [6]:
for company in topList:
    try:
        posts = api.user_timeline(screen_name=company, count =4000, lang ="en", tweet_mode="extended") 
        for eve in posts:
            df = df.append({"Company":company,"Tweets":pre_processing_text(cleanTxt(eve.full_text))},ignore_index=True)
    except Exception:
        pass

In [7]:
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
   return  TextBlob(text).sentiment.polarity

df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

In [8]:
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
df['Analysis'] = df['Polarity'].apply(getAnalysis)

In [9]:
df.to_pickle('Data/final_df')

In [10]:
df2 = pd.DataFrame(columns=['News','Company'])

In [11]:
with open('Data/valid.txt', 'r',encoding="utf-8") as f: 
    for ln in f.readlines():
        df2 = df2.append({"Company":ln.split()[3],"News":pre_processing_text(cleanTxt(ln[23:]))},ignore_index=True)

In [12]:
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
   return  TextBlob(text).sentiment.polarity

df2['Subjectivity'] = df2['News'].apply(getSubjectivity)
df2['Polarity'] = df2['News'].apply(getPolarity)

In [13]:
def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
df2['Analysis'] = df2['Polarity'].apply(getAnalysis)

In [14]:
df2.to_pickle('Data/news_final_df')

In [15]:
df2

,News,Company,Subjectivity,Polarity,Analysis
0,ftse russell puts indian secs on watch for pos...,Indian,1.000000,0.0,Neutral
1,markets follow asian peers gain 2 3,Asian,0.000000,0.0,Neutral
2,hdfc securities users face stock quote mismatch,HDFC,0.000000,0.0,Neutral
3,hdfc securities users face stock quote mismatch,HDFC,0.000000,0.0,Neutral
4,hdfc securities users face stock quote mismatch,HDFC,0.000000,0.0,Neutral
...,...,...,...,...,...
9866,axis bank board approves reclassification of u...,Axis,0.066667,0.0,Neutral
9867,tarun bajaj given additional charge of revenue...,Bajaj,0.000000,0.0,Neutral
9868,tarun bajaj given additional charge of revenue...,Bajaj,0.000000,0.0,Neutral
9869,tarun bajaj given additional charge of revenue...,Bajaj,0.000000,0.0,Neutral
